In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ramil\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ramil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
from nltk.stem import PorterStemmer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [4]:
df = pd.read_csv('labeledTrainData.tsv', sep='\t', encoding='utf-8')

In [5]:
df.to_dict('records')[0]

{'id': '5814_8',
 'sentiment': 1,
 'review': "With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />Th

In [6]:
texts = df.review.tolist()
sentiment = df.sentiment.tolist()

In [7]:
amount = 1000
sentences_train, sentences_test, y_train, y_test = train_test_split(texts[:amount], sentiment[:amount])

In [8]:
import gensim
from bs4 import BeautifulSoup
import re, string
clean_texts = [ ]

for doc in texts:
    x = doc.lower()                     
    x = BeautifulSoup(x, 'lxml').text   
    x = re.sub('[^A-Za-z0-9]+', ' ', x)
    clean_texts.append(x)

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=0, lowercase=False)
vectorizer.fit(clean_texts)

CountVectorizer(lowercase=False, min_df=0)

In [10]:
words_amount = len(vectorizer.vocabulary_)
print(words_amount)

75498


In [11]:
vectorizer.transform(texts).toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [12]:
vectorizer = CountVectorizer()
vectorizer.fit(sentences_train)

X_train = vectorizer.transform(sentences_train)
X_test = vectorizer.transform(sentences_test)
X_train

<750x15933 sparse matrix of type '<class 'numpy.int64'>'
	with 104327 stored elements in Compressed Sparse Row format>

In [11]:
# LogisticRegression
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

print("Accuracy: ", score)

Accuracy:  0.828


c:\users\ramil\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [13]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=words_amount)
tokenizer.fit_on_texts(sentences_train)

X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)
vocab_size = len(tokenizer.word_index) + 1

In [14]:
print(sentences_train[2])
print(X_train[2])

I saw the last five or ten minutes of this film back in 1998 or 1999 one night when I was channel-surfing before going to bed, and really liked what I saw. Since then I've been on the lookout, scouring TV listings, flipping through DVD/VHS racks at stores, but didn't find a copy until recently when I found out some Internet stores sold it. Then, being a world-class procrastinator, I still didn't order it. Finally, I found a DVD copy in a Circuit City while visiting Portland, OR, a few weeks ago. Then it only took me about a month after returning home before sitting down and watching it.<br /><br />So, what do I think about the film? It's good. Not as good as I remembered and hoped for, but still well worth the $9.99 it cost me. After seeing the whole film for the first time I rate it as a 7/10, with potential to become an 8/10. I'll have to be less sleepy then, and have a better sound system to avoid rewinding to catch some dialogue.
[10, 197, 1, 222, 695, 41, 845, 203, 4, 11, 18, 166,

In [19]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [23]:
print(X_train[0])

[  22  418    5    1 2221   57    1 4363 3506    6  554   59  172    1
  411  368 2944  174   27   25  111   78    5 2000  372   17   38 4364
   66   12  571   13  112    1  116  134 8437    6    1  347 1024  220
 2001  321  470   32  127   24  319    5  176  158  967   95   79   28
   13   24  359    2   50  555  175   88   21 4365  166  117    6 3507
    3    1  168 3506    7    7 5770 1427  234 5771    1  173 2002   14
 1428  724 1807 4366 4367    1  182   55  127   24  516    1   19   16
   30   12]


In [24]:
from keras.models import Sequential
from keras import layers
embedding_dim = 50

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 50)           841050    
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 50)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                510       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 11        
Total params: 841,571
Trainable params: 841,571
Non-trainable params: 0
_________________________________________________________________


In [30]:
import numpy as np
y_train = np.array(y_train)
y_test = np.array(y_test)

In [32]:
history = model.fit(X_train, y_train,
                    epochs=20,
                    verbose=False,
                    validation_data=(X_test, y_test),
                    batch_size=10)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 1.0000
Testing Accuracy:  0.7560


In [37]:
embedding_dim = 100

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 100, 100)          1682100   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 96, 128)           64128     
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 10)                1290      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 11        
Total params: 1,747,529
Trainable params: 1,747,529
Non-trainable params: 0
_________________________________________________________________


In [39]:
history = model.fit(X_train, y_train,
                    epochs=40,
                    verbose=False,
                    validation_data=(X_test, y_test),
                    batch_size=10)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


Training Accuracy: 1.0000
Testing Accuracy:  0.7040
